In [1]:
%matplotlib inline
from utils import preprocess

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
DATA_PATH = os.path.join('../data')
TRAIN_FILE = os.path.join(DATA_PATH, 'train.csv')
TEST_FILE = os.path.join(DATA_PATH, 'test.csv')

In [4]:
train_features = preprocess(TRAIN_FILE)
test_features = preprocess(TEST_FILE)

In [5]:
train_features = train_features[['vendor_id', 'passenger_count',
       'trip_duration', 'pickup_hr', 'pickup_min', 'pickup_sec',
       'pickup_day', 'pickup_date', 'pickup_mon',
       's_pickup_latitude', 's_pickup_longitude', 's_dropoff_latitude',
       's_dropoff_longitude', 'flag', 'cityblock']]

In [6]:
test_features = test_features[['vendor_id', 'passenger_count','pickup_hr', 'pickup_min', 'pickup_sec',
       'pickup_day', 'pickup_date', 'pickup_mon',
       's_pickup_latitude', 's_pickup_longitude', 's_dropoff_latitude',
       's_dropoff_longitude', 'flag', 'cityblock']]

In [7]:
X_train = train_features[['vendor_id', 'passenger_count',
        'pickup_hr', 'pickup_min', 'pickup_sec',
       'pickup_day', 'pickup_date', 'pickup_mon',
       's_pickup_latitude', 's_pickup_longitude', 's_dropoff_latitude',
       's_dropoff_longitude', 'flag', 'cityblock']].as_matrix()
y_train = train_features['trip_duration']

In [8]:
X_test = test_features.as_matrix()

In [9]:
print X_train.shape
print y_train.shape
print X_test.shape

(1458644, 14)
(1458644,)
(625134, 14)


In [10]:
from sklearn.ensemble import RandomForestRegressor

In [11]:
regressor = RandomForestRegressor(n_estimators=10, criterion='mse',
                                 max_features=None, min_samples_leaf=50, bootstrap=False)

In [12]:
regressor.fit(X_train, y_train)

In [21]:
model.score(X_train, y_train)

0.039410002689560741

In [22]:
predictions = model.predict(X_test)

In [23]:
predictions.shape

(625134,)

In [24]:
from sklearn.metrics import mean_squared_error

In [27]:
np.sqrt(mean_squared_error(y_train, model.predict(X_train)))

5133.1890083549224

In [28]:
model2 = RandomForestRegressor(n_estimators=100, criterion='mse',
                                 max_features=None).fit(X_train, y_train)

KeyboardInterrupt: 

In [29]:
dir(model)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__hash__',
 '__init__',
 '__iter__',
 '__len__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_estimator_type',
 '_get_param_names',
 '_make_estimator',
 '_set_oob_score',
 '_validate_X_predict',
 '_validate_estimator',
 '_validate_y_class_weight',
 'apply',
 'base_estimator',
 'base_estimator_',
 'bootstrap',
 'class_weight',
 'criterion',
 'decision_path',
 'estimator_params',
 'estimators_',
 'feature_importances_',
 'fit',
 'fit_transform',
 'get_params',
 'max_depth',
 'max_features',
 'max_leaf_nodes',
 'min_impurity_split',
 'min_samples_leaf',
 'min_samples_split',
 'min_weight_fraction_leaf',
 'n_estimators',
 'n_feat

In [32]:
dir(model.estimators_[0])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getstate__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_estimator_type',
 '_get_param_names',
 '_validate_X_predict',
 'apply',
 'class_weight',
 'classes_',
 'criterion',
 'decision_path',
 'feature_importances_',
 'fit',
 'fit_transform',
 'get_params',
 'max_depth',
 'max_features',
 'max_features_',
 'max_leaf_nodes',
 'min_impurity_split',
 'min_samples_leaf',
 'min_samples_split',
 'min_weight_fraction_leaf',
 'n_classes_',
 'n_features_',
 'n_outputs_',
 'predict',
 'presort',
 'random_state',
 'score',
 'set_params',
 'splitter',
 'transform',
 'tree_']